In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import pickle

# w2v_embeddings = KeyedVectors.load("word2vec_first_embeddings")
# w2v_model = Word2Vec.load("en_1000_no_stem/en.model")

In [2]:
y_train, x_b, x_train = pickle.load(open("dbpedia_csv/" + "/train_vec_preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, x_tb, x_test = pickle.load(open("dbpedia_csv/" + "/test_vec_preprocessed_vectorized.pickle", "rb"))

In [3]:
split_examples =[x.split() for x in x_train]

In [4]:
split_test = [x.split() for x in x_test]

In [3]:
import pandas
import numpy

In [6]:
def word_vector(tokens, size):
    vec = numpy.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [7]:
wordvec_arrays = numpy.zeros((len(split_examples), 1000)) 
for i in range(len(split_examples)):
    wordvec_arrays[i,:] = word_vector(split_examples[i], 1000)

In [8]:
wordvec_df = pandas.DataFrame(wordvec_arrays)
wordvec_df.shape

(560000, 1000)

In [9]:
x_train = wordvec_df

In [10]:
wordvec_arrays = numpy.zeros((len(split_test), 1000)) 
for i in range(len(split_test)):
    wordvec_arrays[i,:] = word_vector(split_test[i], 1000)
    
wordvec_df = pandas.DataFrame(wordvec_arrays)

x_test = wordvec_df
x_test.shape

(70000, 1000)

In [11]:
from preprocess import preprocess_list, write_pkl

In [12]:
write_pkl(y_train, x_b, x_train, mode="train_vec")

In [13]:
write_pkl(y_test, x_tb, x_test, mode="test_vec")

In [4]:
y_train = [y-1 for y in y_train]
y_train[:5]

[0, 0, 0, 0, 0]

In [5]:
import xgboost as xgb

from sklearn.metrics import classification_report, confusion_matrix

In [19]:
params = {
    'max_depth': 6,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 14,
    # Set number of GPUs if available   
    'n_gpus': 2
}

In [6]:
dtrain = xgb.DMatrix(data=x_train, label=y_train)
dtest = xgb.DMatrix(data=x_test)


In [ ]:
bst = xgb.train(params, dtrain)

In [ ]:
pred = bst.predict(dtest)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:


cm = confusion_matrix(y_test, pred)
cm



In [12]:
y_train[-1]

13

In [13]:
y_test = [y-1 for y in y_test]

In [7]:
from xgboost import XGBClassifier
from time import time


params = {
    'max_depth': 6,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 14,
    # Set number of GPUs if available   
    'tree_method': 'gpu_hist',
    'gpu_id': 0
}
t = time()
bst = xgb.train(params, dtrain)

training_time = time() - t
print("train time: %0.3fs" % training_time)


t = time()
pred = bst.predict(dtest)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

print(classification_report(y_test, pred))
cm = confusion_matrix(y_test, pred)
cm

train time: 82.990s
test time:  0.265s
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.01      0.01      0.01      5000
         2.0       0.00      0.00      0.00      5000
         3.0       0.01      0.01      0.01      5000
         4.0       0.00      0.00      0.00      5000
         5.0       0.00      0.00      0.00      5000
         6.0       0.00      0.00      0.00      5000
         7.0       0.01      0.01      0.01      5000
         8.0       0.00      0.00      0.00      5000
         9.0       0.00      0.00      0.00      5000
        10.0       0.03      0.03      0.03      5000
        11.0       0.00      0.00      0.00      5000
        12.0       0.01      0.01      0.01      5000
        13.0       0.02      0.02      0.02      5000
        14.0       0.00      0.00      0.00      5000

    accuracy                           0.01     70000
   macro avg       0.01      0.01      0.

C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classificat

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [4269,   51,   84,   19,   44,  165,   86,   15,    1,    6,   11,
          55,   34,  160,    0],
       [  60, 4732,   21,   13,   53,    6,   69,    8,    5,    1,    4,
           0,    7,   21,    0],
       [  92,   26, 4058,   30,  106,    7,   28,    2,    0,    2,    5,
         190,   99,  355,    0],
       [   7,    7,   60, 4867,   16,   23,    6,    0,    0,    3,    1,
           0,    3,    7,    0],
       [  37,   60,  130,   22, 4672,   20,   18,    3,    1,    2,    0,
           0,    7,   28,    0],
       [ 148,    3,    4,    4,    9, 4781,   24,    9,    0,    4,    0,
           1,    3,   10,    0],
       [  90,   90,   29,    6,   43,   53, 4598,   63,   11,    2,    1,
           0,    4,   10,    0],
       [   2,    7,    2,    1,    8,   16,   56, 4864,   17,   10,    8,
           0,    2,    7,    0],
       [   3,    2,    2,    1,    9,

In [20]:
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import precision_score

In [14]:
score1 = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score1)

accuracy:   0.932


In [15]:
print(classification_report(y_test, pred))
cm = confusion_matrix(y_test, pred)
cm

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      5000
           1       0.95      0.95      0.95      5000
           2       0.84      0.81      0.83      5000
           3       0.97      0.97      0.97      5000
           4       0.93      0.93      0.93      5000
           5       0.94      0.96      0.95      5000
           6       0.93      0.92      0.92      5000
           7       0.97      0.97      0.97      5000
           8       0.99      0.98      0.99      5000
           9       0.96      0.96      0.96      5000
          10       0.97      0.96      0.97      5000
          11       0.94      0.96      0.95      5000
          12       0.94      0.95      0.94      5000
          13       0.86      0.86      0.86      5000

    accuracy                           0.93     70000
   macro avg       0.93      0.93      0.93     70000
weighted avg       0.93      0.93      0.93     70000



array([[4269,   51,   84,   19,   44,  165,   86,   15,    1,    6,   11,
          55,   34,  160],
       [  60, 4732,   21,   13,   53,    6,   69,    8,    5,    1,    4,
           0,    7,   21],
       [  92,   26, 4058,   30,  106,    7,   28,    2,    0,    2,    5,
         190,   99,  355],
       [   7,    7,   60, 4867,   16,   23,    6,    0,    0,    3,    1,
           0,    3,    7],
       [  37,   60,  130,   22, 4672,   20,   18,    3,    1,    2,    0,
           0,    7,   28],
       [ 148,    3,    4,    4,    9, 4781,   24,    9,    0,    4,    0,
           1,    3,   10],
       [  90,   90,   29,    6,   43,   53, 4598,   63,   11,    2,    1,
           0,    4,   10],
       [   2,    7,    2,    1,    8,   16,   56, 4864,   17,   10,    8,
           0,    2,    7],
       [   3,    2,    2,    1,    9,    2,   35,   44, 4896,    2,    1,
           0,    0,    3],
       [   3,    0,    4,   10,    2,    4,    4,    9,    0, 4825,  133,
           0,    

In [16]:
import lightgbm as lgb

In [22]:
d_train=lgb.Dataset(x_train, label=y_train)
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_leaves'] = 1100
params['num_class']=14 #no.of unique values in the target class not inclusive of the end value
params['device'] = 'gpu'
t = time()
clf=lgb.train(params,d_train,100)
training_time = time() - t
print("train time: %0.3fs" % training_time)


t = time()
y_pred_1=clf.predict(x_test)

test_time = time() - t
print("test time:  %0.3fs" % test_time)





[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 255000
[LightGBM] [Info] Number of data points in the train set: 560000, number of used features: 1000
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1050 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1000 dense feature groups (534.06 MB) transferred to GPU in 0.354069 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [Info] Start training from score -2.639057
[LightGBM] [In

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [23]:

y_pred_1 = [numpy.argmax(line) for line in y_pred_1]#printing the predictions

print(precision_score(y_pred_1,y_test,average=None).mean())

print(classification_report(y_test, y_pred_1))
cm = confusion_matrix(y_test, y_pred_1)
cm

0.9566285714285714
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      5000
           1       0.96      0.96      0.96      5000
           2       0.89      0.88      0.89      5000
           3       0.98      0.98      0.98      5000
           4       0.95      0.96      0.95      5000
           5       0.96      0.97      0.97      5000
           6       0.95      0.95      0.95      5000
           7       0.98      0.98      0.98      5000
           8       1.00      0.98      0.99      5000
           9       0.98      0.98      0.98      5000
          10       0.99      0.98      0.98      5000
          11       0.96      0.97      0.97      5000
          12       0.96      0.96      0.96      5000
          13       0.90      0.92      0.91      5000

    accuracy                           0.96     70000
   macro avg       0.96      0.96      0.96     70000
weighted avg       0.96      0.96      0.96     70000



array([[4523,   38,   55,   13,   27,  104,   63,    4,    2,    5,    3,
          24,   23,  116],
       [  53, 4823,   14,    6,   31,    2,   47,    6,    4,    0,    2,
           0,    2,   10],
       [  65,   14, 4410,   14,   82,    5,   13,    1,    0,    2,    4,
         122,   48,  220],
       [   8,    2,   46, 4912,   15,    7,    4,    0,    0,    3,    0,
           0,    1,    2],
       [  31,   36,   91,   14, 4778,   14,   14,    1,    0,    1,    0,
           0,    2,   18],
       [  88,    1,    5,    5,    5, 4865,   16,    5,    1,    3,    0,
           0,    4,    2],
       [  73,   64,   14,    3,   29,   21, 4738,   40,    5,    2,    0,
           0,    3,    8],
       [   5,    1,    1,    0,    5,    9,   41, 4917,   12,    5,    3,
           0,    0,    1],
       [   4,    2,    2,    0,    7,    2,   29,   30, 4921,    2,    0,
           0,    0,    1],
       [   3,    0,    4,    7,    2,    1,    3,    7,    0, 4915,   56,
           0,    